# 指数定投策略

## 策略1：每天等额定投

每天等额定投，拉长时间至3~5年看，年化收益率能有多少。

使用的工具为JoinQuant: https://github.com/JoinQuant/jqdatasdk

考察的指数有：([指数列表](https://www.joinquant.com/data/dict/indexData))
- 沪深300：000300.XSHG
- 上证50：000016.XSHG
- 中证500：399905.XSHE

指数相关API：https://www.joinquant.com/help/api/help#name:index

In [1]:
import jqdatasdk
import math
import numpy as np
from matplotlib import pyplot as plt
# ------ 这里需要替换为你自己在JoinQuant里的用户名和密码！ -----
jqdatasdk.auth("UserName", "PassWord")

# 隐藏警告
import warnings
warnings.filterwarnings('ignore')

auth success 


---

以沪深300为例：

In [9]:
def calc_annual_yield(index_code, start_date, end_date, n): # n: 年数
    df = jqdatasdk.get_price(index_code, start_date=start_date, end_date=end_date, frequency='daily', fields=["close"], skip_paused=False, fq='pre')
    # print(df)

    close_price_list = [data[0] for idx, data in df.iterrows()]
    
    shares = 0
    for i in range(len(close_price_list)):
        shares += 1000.0 / close_price_list[i] # 每天以收盘价买入1000元
    
    asset = close_price_list[-1] * shares
    cost = 1000.0 * len(close_price_list)
    
    total_yield = 1.0 * (asset - cost) / cost
    # print(total_yield)
    
    # total_yield = (1 + annual_yield) ^ n - 1
    annual_yield = math.pow(total_yield + 1, 1.0 / n) - 1.0
    return annual_yield * 100

In [3]:
calc_annual_yield("000300.XSHG", '2015-01-01', '2015-12-31', 1)

-3.3110765462758818

从2006年开始，到2016年，分别计算定投沪深300 1-5年的收益率矩阵。

In [6]:
def calc_yield_matrix(security_code):
    for start_year in range(2006, 2021):
        yields = []
        for delta in range(16):
            end_year = start_year + delta
            if end_year == 2021:
                break
            y = calc_annual_yield(security_code, '{}-01-01'.format(start_year), '{}-12-31'.format(end_year), delta + 1)
            yields.append(y)
        s = "start from {}" + " | {:.2f}" * len(yields)
        print(s.format(start_year, *yields))

In [7]:
calc_yield_matrix("000300.XSHG")

start from 2006 | 59.88 | 67.54 | -5.06 | 12.01 | 5.49 | -0.83 | 0.37 | -0.66 | 4.19 | 3.94 | 2.27 | 3.55 | 0.83 | 2.86 | 4.16
start from 2007 | 43.80 | -24.17 | 5.38 | 0.58 | -4.95 | -2.31 | -2.72 | 3.17 | 3.06 | 1.41 | 2.92 | 0.09 | 2.36 | 3.80
start from 2008 | -34.08 | 12.81 | 2.70 | -5.09 | -1.94 | -2.54 | 4.16 | 3.82 | 1.84 | 3.43 | 0.26 | 2.68 | 4.20
start from 2009 | 24.82 | 2.83 | -7.24 | -2.47 | -2.98 | 4.98 | 4.37 | 2.04 | 3.77 | 0.22 | 2.86 | 4.47
start from 2010 | 2.24 | -11.25 | -2.99 | -3.39 | 6.34 | 5.22 | 2.38 | 4.23 | 0.21 | 3.10 | 4.82
start from 2011 | -19.17 | -2.18 | -3.17 | 8.98 | 6.72 | 3.01 | 4.97 | 0.29 | 3.47 | 5.30
start from 2012 | 4.48 | -1.96 | 13.85 | 8.96 | 3.81 | 5.85 | 0.30 | 3.85 | 5.82
start from 2013 | -4.27 | 21.73 | 10.90 | 3.87 | 6.25 | -0.27 | 3.86 | 6.06
start from 2014 | 51.08 | 13.21 | 3.25 | 6.38 | -1.35 | 3.68 | 6.23
start from 2015 | -3.31 | -2.91 | 4.24 | -4.11 | 2.65 | 5.88
start from 2016 | 2.75 | 8.48 | -4.63 | 3.77 | 7.27
start from 

In [10]:
calc_yield_matrix("000016.XSHG")

start from 2006 | 73.16 | 65.40 | -6.75 | 9.27 | 1.43 | -2.22 | 0.25 | -1.73 | 4.20 | 2.79 | 1.87 | 3.43 | 1.26 | 3.12 | 3.94
start from 2007 | 42.25 | -25.49 | 2.87 | -3.43 | -5.84 | -1.89 | -3.50 | 3.54 | 2.10 | 1.23 | 3.02 | 0.74 | 2.80 | 3.70
start from 2008 | -35.71 | 9.63 | -2.13 | -5.76 | -1.14 | -3.24 | 4.77 | 2.87 | 1.76 | 3.64 | 1.05 | 3.23 | 4.15
start from 2009 | 21.85 | -2.67 | -7.12 | -0.88 | -3.46 | 5.99 | 3.51 | 2.14 | 4.14 | 1.21 | 3.55 | 4.51
start from 2010 | -4.89 | -9.66 | -0.28 | -3.68 | 7.78 | 4.37 | 2.61 | 4.76 | 1.38 | 3.92 | 4.91
start from 2011 | -14.60 | 2.00 | -3.62 | 10.81 | 5.69 | 3.28 | 5.57 | 1.60 | 4.36 | 5.38
start from 2012 | 10.02 | -3.68 | 15.89 | 7.36 | 3.97 | 6.43 | 1.69 | 4.75 | 5.81
start from 2013 | -7.72 | 25.32 | 8.96 | 4.23 | 7.04 | 1.37 | 4.90 | 6.06
start from 2014 | 62.65 | 11.04 | 4.10 | 7.57 | 0.68 | 4.91 | 6.23
start from 2015 | -6.06 | -1.46 | 5.78 | -1.76 | 3.94 | 5.74
start from 2016 | 5.47 | 10.48 | -1.85 | 5.08 | 6.88
start from 

In [8]:
calc_yield_matrix("399905.XSHE")

start from 2006 | nan | nan | nan | nan | nan | nan | nan | nan | nan | nan | nan | nan | nan | nan | nan
start from 2007 | nan | nan | nan | nan | nan | nan | nan | nan | nan | nan | nan | nan | nan | nan
start from 2008 | -30.40 | 21.91 | 13.34 | -2.40 | -1.60 | 1.21 | 5.54 | 8.76 | 4.94 | 4.04 | -0.25 | 1.64 | 2.84
start from 2009 | 36.19 | 13.85 | -6.47 | -3.75 | 0.28 | 5.55 | 9.15 | 4.79 | 3.81 | -0.83 | 1.31 | 2.65
start from 2010 | 9.18 | -14.50 | -6.68 | -0.59 | 5.96 | 9.96 | 4.82 | 3.70 | -1.39 | 1.04 | 2.53
start from 2011 | -26.05 | -7.45 | 0.66 | 8.29 | 12.27 | 5.66 | 4.17 | -1.63 | 1.10 | 2.72
start from 2012 | -2.81 | 4.77 | 12.72 | 15.81 | 6.74 | 4.70 | -2.01 | 1.11 | 2.90
start from 2013 | 5.86 | 16.55 | 18.03 | 6.11 | 3.89 | -3.53 | 0.35 | 2.50
start from 2014 | 24.85 | 19.15 | 3.75 | 2.03 | -5.89 | -0.77 | 1.93
start from 2015 | 5.11 | -3.05 | -1.49 | -9.25 | -2.12 | 1.36
start from 2016 | 1.56 | 0.12 | -10.68 | -1.51 | 2.39
start from 2017 | -0.85 | -14.50 | -1.01 | 